In [1]:
import psycopg2

import pandas as pd
import numpy as np

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from copy import deepcopy

import re, unicodedata

from joblib import Parallel, delayed
# from graph_tool.all import *

### parse sehir contacts directory (full names, emails)

In [2]:
sehir_directory = pd.read_csv('datasets/contacts.csv', 
                               encoding = "ISO-8859-1", 
                               usecols=['First Name', 'Last Name', 'Primary Email']).dropna()

In [3]:
sehir_directory.head()

,First Name,Last Name,Primary Email
102,Burak,Fidan,burakfidan@std.sehir.edu.tr
103,Muhammed,Aydin,cihadaydin@std.sehir.edu.tr
104,Muhammet Masuk,Aktas,muhammetaktas@std.sehir.edu.tr
105,Sezer,Aktas,sezeraktas@std.sehir.edu.tr
106,Ayse Nur,Salk,aysesalk@std.sehir.edu.tr


In [4]:
"{} sehir contacts".format(len(sehir_directory))

'9088 sehir contacts'

In [5]:
fullnames = [' '.join(first_last_name).lower() 
                 for first_last_name in sehir_directory[['First Name', 'Last Name']].values]

In [6]:
def get_matches_edit_distance(item, choices, limit, scorer=fuzz.token_sort_ratio):
    return process.extract(item, choices, limit=limit, scorer=scorer)
counter = 0
def get_sehir_twitter_matches(twitter_users, sehir_directory, limit=2):
    global fullnames,twitter_users_count
    twitter_user_by_screen_name = twitter_users.set_index('twitter_screen_name')
    
    for twitter_screen_name in twitter_users['twitter_screen_name']:
        twitter_name = twitter_user_by_screen_name.loc[twitter_screen_name]['cleaned_twitter_name']
        sehir_matches = get_matches_edit_distance(twitter_name, fullnames, limit)
        counter += 1
        if counter %100 == 0:
            print(counter, "out of ", twitter_users_count)
        yield (twitter_screen_name, sehir_matches)
        
def filter_matches_by_threshold(matches_dict, threshold=70):
    filtered_dict = dict()
    for twitter_screen_name, matches in matches_dict.items():
        filtered = [(match, score) for match, score in matches if score > threshold]
        
        if filtered:
            filtered_dict[twitter_screen_name] = filtered
        
    return filtered_dict

def get_matches_dataframe(twitter_users, sehir_directory, threshold, limit):
    matches = {screen_name : sehir_matches for screen_name, sehir_matches in 
               get_sehir_twitter_matches(twitter_users, sehir_directory, limit=limit)}
    
    filtered_matches = filter_matches_by_threshold(matches, threshold=threshold)
    screen_names = filtered_matches.keys()
    return pd.DataFrame({'twitter_screen_name': list(screen_names),
                         'sehir_matches': [filtered_matches[screen_name] for screen_name in screen_names]})

def clean(name):
    return unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').lower().decode("ascii")

### connect to sql database

In [7]:
connection = psycopg2.connect('dbname=twitter_accounts_new host=localhost user=postgres password=1_sehir_1')

twitter_users = pd.read_sql("SELECT * FROM twitter_user", connection)\
.rename(columns={'id': 'GUID', 
                 'name': 'twitter_name',
                 'description': 'profile_description',
                 'screen_name': 'twitter_screen_name'})

user_connections = pd.read_sql("SELECT * FROM twitter_connection", connection).drop('id', axis=1)

In [8]:
twitter_users["cleaned_twitter_name"] = twitter_users["twitter_name"].apply(lambda x:clean(x) if len(clean(x))>4 else np.NaN)
twitter_users.dropna(inplace=True)
twitter_users.head(5)

,GUID,twitter_name,twitter_screen_name,profile_description,followers_count,friends_count,favourites_count,statuses_count,lang,cleaned_twitter_name
0,2924769794,Elif Öztürk,bidikshowtime,"Çanakkale Fen Lisesi'nde okumuş, Boğaziçi Üniv...",46,146,1,2,en,elif ozturk
2,78362287,Ozan Kublay,ozankublay,,307,892,1093,69,en,ozan kublay
3,927559556,Dilay Kirişçi,annebanafalbak,,119,146,34,307,tr,dilay kirisci
4,567346178,Aslı Atay,asliatayy,,156,523,69,1687,tr,asl atay
5,94153120,Özgecan,theewayfarer,Student athlete and research assistant,136,472,1097,1513,en,ozgecan


In [9]:
twitter_users_count = len(twitter_users)
twitter_users_count

34142

### user_connections contains follower/followee relationships by GUID

In [10]:
user_connections.head(2)

,from_user_id,to_user_id
0,2924769794,566810673
1,2924769794,78362287


### matching subset of sehir twitter accounts _ALL_

In [11]:
sehir_matches_df = get_matches_dataframe(twitter_users,
                                         sehir_directory, threshold=80, limit=2)

UnboundLocalError: local variable 'counter' referenced before assignment

In [ ]:
print("There are {} matches".format(len(sehir_matches_df)))
sehir_matches_df.head(5)

In [ ]:
merged = sehir_matches_df.merge(twitter_users, left_on='twitter_screen_name', right_on='twitter_screen_name')

In [ ]:
merged[['GUID', 'twitter_screen_name', 'cleaned_twitter_name', 'sehir_matches']].tail(5)

In [ ]:
merged.to_csv('datasets/sehir_matches.csv', index_label="GUID")

In [ ]:
merged = pd.read_csv('datasets/sehir_matches.csv', index_col="GUID")

In [ ]:
merged.head(5)

In [ ]:
# merged.set_index('GUID.1').loc[291122559]

In [ ]:
'{} twitter accounts have been matched successfully'.format(len(merged))

In [ ]:
# sehir_matches_by_guid = merged.set_index('GUID')

In [ ]:
number_of_matches, _ = merged.shape

network = Graph(directed=True)
network.add_vertex(number_of_matches)

vprop = network.new_vertex_property("string")

for vertex, twitter_screen_name in zip(network.vertices(), merged['twitter_screen_name'].values):
    vprop[vertex] = twitter_screen_name
    
network.vertex_properties["twitter_screen_name"] = vprop

for i, row in user_connections.iterrows():
    from_ = row['from_user_id']
    to = row['to_user_id']
    
    try:
        network.add_edge(find_vertex(network, vprop, sehir_matches_by_guid.loc[from_]['twitter_name']), 
                         find_vertex(network, vprop, sehir_matches_by_guid.loc[to]['twitter_name']))
    except Exception as e:
        pass

In [ ]:
graph_draw(network, vertex_text=network.vertex_properties["twitter_screen_name"],
           vertex_font_size=40,
           output_size=(10000, 10000), output="sehir_subset.png")